In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

## Introduction

The purpose of this notebook is to try out the image recognition techniques introduced in the first lecture of 'Practical Deep Learning for Coders'. Our dataset will consist of images of Lego's. The dataset was attained from Kaggle: https://www.kaggle.com/joosthazelzet/lego-brick-images

The dataset itself is downloaded with Kaggle's CLI tool. In the data folder one can run the following command to download the dataset: `kaggle datasets download -d joosthazelzet/lego-brick-images
`. 

In [12]:
# The path to the data dir
data_dir_raw = os.path.abspath(
    os.path.join(
        os.getcwd(), '..', '..', 'data', 'raw', 'LEGO brick images'
    )
)